In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.api as sm
import pulp
from scipy.optimize import minimize

import opendatasets as od
from IPython.core.display import display, HTML
display(HTML("<style>.container { width : 98% !important; }</style>"))

pd.set_option('display.max_columns',500)

In [2]:
# Import all classes of PuLP module
from pulp import *

# Create the problem variable to contain the problem data
model = LpProblem("ProductionProblem", LpMaximize)

# Create 3 variables tables, chairs, and bookcases

foldy = LpVariable("foldy", 0, None, LpContinuous) # takes 1.5 hrs to make a foldy phone
tiny = LpVariable("tiny", 0, None,  LpContinuous)  # takes 2 hrs to make a tiny phone
LpVariable()
# Create maximize objective function
model += 900 * foldy + 1100 * tiny

# Create the constraints
model += 2 * tiny + 1.5 * foldy <= 2999.5, "labor"
model += tiny >= 200, "tinyphone_min"
model += foldy >= 500, "foldyphone_min"

# The problem is solved using PuLP's choice of Solver
model.solve()

# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/fe6b605f4ac8445f91ac4782ea8086fa-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/fe6b605f4ac8445f91ac4782ea8086fa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 15 RHS
At line 19 BOUNDS
At line 20 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1779700
After Postsolve, objective 1779700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1779700 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions

In [3]:
# CAN USE THIS ONE BELOW, BUT THE LOOPS MAKE IT HARDER TO DISSECT

In [6]:
from pulp import *
def define_problem_objective(steak_price: float, peanut_butter_price: float):
    problem = LpProblem(name="DietProblem", sense=LpMinimize)
    S = LpVariable(name = "Steak_In_Pounds", lowBound = 0, cat = LpContinuous)
    P = LpVariable(name = "Peanut_Butter_In_Pounds", lowBound = 0, cat = LpContinuous)
    problem += (steak_price*S) + (peanut_butter_price * P), "MinimizeCost"
    return (problem, S, P)

def add_constraints(problem: LpProblem, S: LpVariable, P: LpVariable):
    problem += (2*S) + P >= 4, "MinimumProteinIntake"
    problem += S >= 0, "MinimumSteakQuantity"
    problem += P >= 0, "MinimumPeanutButterQuantity"
    
prices = [(3,2), (4,2), (4,2.5), (12, 2), (10,3)]
for steak_price, peanut_butter_price in prices:
    print("\n****************************************************")
    print(f"Steak Price: {steak_price} | Peanut Butter Price: {peanut_butter_price}")
    problem, steak, peanut_butter = define_problem_objective(steak_price, peanut_butter_price)
    add_constraints(problem, steak, peanut_butter)
    problem.solve(PULP_CBC_CMD(msg=0))
    print(f"Status of solution is: {LpStatus[problem.status]}")
    print(f"Optimal cost per day is: ${problem.objective.value()}")
    print(f"Steak (in pounds): {steak.value()}")
    print(f"Peanut Butter (in pounds): {peanut_butter.value()}")


****************************************************
Steak Price: 3 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $6.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 4 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 4 | Peanut Butter Price: 2.5
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 12 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 0.0
Peanut Butter (in pounds): 4.0

****************************************************
Steak Price: 10 | Peanut Butter Price: 3
Status of solution is: Optimal
Optimal cost per day is: $12.0
Stea

In [14]:
# Simplest example without constraints or multiple variables
def f(x):
    y = (x**2) - (12 * x) + 20
    return y

x_start = 2.0

result = scipy.optimize.minimize(f,x_start)
# Can also do the below to watch it optimize:
# result = scipy.optimize.minimize(f, x_start, options = {"disp":True})

if result.success:
    print(f"x={result.x} y = {result.fun}")
else:
    print('Could not optimize')

x=[6.00000025] y = -15.999999999999936


In [15]:
# Simplest example without constraints or multiple variables
def f(xy):
    x = xy[0]
    y = xy[1]
    area = x * y
    return -area

xy_start = [50,50]

cons = ({'type':'eq','fun':lambda xy : (2*xy[0] + xy[1] - 100)})

bnds = ((1,100),(1,100))

result = scipy.optimize.minimize(f, x_start, constraints=cons, bounds=bnds)
# Can also do the below to watch it optimize:
# result = scipy.optimize.minimize(f, x_start, options = {"disp":True})

if result.success:
    xy = result.x
    x = xy[0]
    y = xy[1]
    print(f"x = {x} y = {y}")
else:
    print('Could not optimize')

x = 25.000005089367114 y = 49.99998982126577
